# Notebook 5 – Koide Playground

**Level:** Beginner  
**Topics:** Koide relation, lepton mass pattern, geometric angles, simple interactivity

> **Provenance:** This notebook is based on:
> - `examples/koide_analysis.py`
> - `light_theory_realm/pocket_u_lite/koide.py`

In [1]:
# %pip install light-theory-realm ipywidgets matplotlib

import matplotlib.pyplot as plt

# Try both top-level and subpackage imports for robustness
try:
    from light_theory_realm import koide_ratio, koide_angle, predict_third_mass
except ImportError:
    from light_theory_realm.pocket_u_lite import (
        koide_ratio,
        koide_angle,
        predict_third_mass,
    )

print("Koide helpers loaded.")

W1202 15:17:46.377508   91850 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1202 15:17:46.381398   91743 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.


Koide helpers loaded.


## 1. Baseline: PDG lepton masses

We’ll compute:

- The Koide ratio $ Q $
- The Koide angle between $\sqrt{\text{mass}}$ vectors

In [2]:
# PDG-like values in MeV
m_e = 0.51099895
m_mu = 105.6583755
m_tau = 1776.86

Q = koide_ratio(m_e, m_mu, m_tau)
angle_deg = koide_angle(m_e, m_mu, m_tau)["theta_deg"]

print(f"Koide ratio Q(e, μ, τ) = {Q:.9f}")
print(f"Koide angle           = {angle_deg:.6f}° (target ≈ 45°)")
print(f"Deviation from 2/3    = {abs(Q - 2/3):.3e}")

Koide ratio Q(e, μ, τ) = 0.666660511
Koide angle           = 44.999735° (target ≈ 45°)
Deviation from 2/3    = 6.155e-06


In [3]:
try:
    import ipywidgets as widgets
    from ipywidgets import interact
    HAS_WIDGETS = True
except ImportError:
    HAS_WIDGETS = False
    print("ipywidgets not installed; using manual exploration instead.")

ipywidgets not installed; using manual exploration instead.


In [4]:
if HAS_WIDGETS:
    @interact(
        m_e=widgets.FloatLogSlider(value=m_e, base=10, min=-1, max=1, step=0.01, description="m_e [MeV]"),
        m_mu=widgets.FloatLogSlider(value=m_mu, base=10, min=0, max=3, step=0.01, description="m_mu [MeV]"),
        m_tau=widgets.FloatLogSlider(value=m_tau, base=10, min=0, max=4, step=0.01, description="m_tau [MeV]"),
    )
    def koide_play(m_e=m_e, m_mu=m_mu, m_tau=m_tau):
        Q = koide_ratio(m_e, m_mu, m_tau)
        angle_deg = koide_angle(m_e, m_mu, m_tau)["theta_deg"]
        Q_error = abs(Q - 2/3)

        print(f"m_e  = {m_e:.6f} MeV")
        print(f"m_mu = {m_mu:.6f} MeV")
        print(f"m_tau= {m_tau:.6f} MeV")
        print()
        print(f"Koide Q      = {Q:.9f}")
        print(f"Koide angle  = {angle_deg:.6f}°")
        print(f"|Q - 2/3|    = {Q_error:.3e}")

In [5]:
if not HAS_WIDGETS:
    def koide_report(m_e, m_mu, m_tau):
        Q = koide_ratio(m_e, m_mu, m_tau)
        angle_deg = koide_angle(m_e, m_mu, m_tau)["theta_deg"]
        Q_error = abs(Q - 2/3)

        print(f"m_e  = {m_e:.6f} MeV")
        print(f"m_mu = {m_mu:.6f} MeV")
        print(f"m_tau= {m_tau:.6f} MeV")
        print()
        print(f"Koide Q      = {Q:.9f}")
        print(f"Koide angle  = {angle_deg:.6f}°")
        print(f"|Q - 2/3|    = {Q_error:.3e}")

    # Example usage:
    koide_report(m_e, m_mu, m_tau)

m_e  = 0.510999 MeV
m_mu = 105.658376 MeV
m_tau= 1776.860000 MeV

Koide Q      = 0.666660511
Koide angle  = 44.999735°
|Q - 2/3|    = 6.155e-06


## 2. Predicting the third mass

Given two masses and the Koide ansatz, we can predict the third mass by solving the Koide condition.

In [6]:
def show_third_mass_prediction(m1, m2, label1="m1", label2="m2"):
    pred, _ = predict_third_mass(m1, m2)
    Q = koide_ratio(m1, m2, pred)
    angle_deg = koide_angle(m1, m2, pred)["theta_deg"]

    print(f"{label1} = {m1:.6f} MeV")
    print(f"{label2} = {m2:.6f} MeV")
    print(f"Predicted m3 = {pred:.6f} MeV")
    print(f"Q(m1, m2, m3) = {Q:.9f}")
    print(f"Koide angle   = {angle_deg:.6f}°")

show_third_mass_prediction(m_e, m_mu, "m_e", "m_mu")


m_e = 0.510999 MeV
m_mu = 105.658376 MeV
Predicted m3 = 1776.969027 MeV
Q(m1, m2, m3) = 0.666666667
Koide angle   = 45.000000°


## 3. Are we within X% of Q = 2/3?

Define a simple helper that checks whether a mass triple satisfies the Koide condition within a tolerance.

In [7]:
def within_tol(m1, m2, m3, tol_percent=0.1):
    Q = koide_ratio(m1, m2, m3)
    rel_err = abs(Q - 2/3) / (2/3) * 100.0
    return Q, rel_err, (rel_err <= tol_percent)

Q_val, rel_err, ok = within_tol(m_e, m_mu, m_tau, tol_percent=0.1)

print(f"Q(e, μ, τ)   = {Q_val:.9f}")
print(f"Rel error    = {rel_err:.4f}%")
print(f"Within 0.1%? = {ok}")

Q(e, μ, τ)   = 0.666660511
Rel error    = 0.0009%
Within 0.1%? = True


## 4. Next steps

- Try plugging in **quark masses** to see how badly they fail Koide.
- Scan over a range of hypothetical third masses and plot **Q vs m₃**.
- Continue to **Notebook 6 – Geometric Fingerprints of Particles** to connect Koide-like structure with geometric diagnostics (Fisher, Berry, ξ).